# Data Transformation

In this notebook we'll look at how to transform the input datasets to be consumed into a Data Model.

### Step 1 - What inputs do you have?

In [1]:
%%bash
ls ../data

WDICountry.csv
WDIData.csv
WDISeries.csv


### Step 2 - Get your tools setup

As part of this we will be using [PySpark](http://spark.apache.org/docs/2.1.1/api/python/index.html) to inspect the data on hand and also gather some basic details.

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd

#Locating where pyspark is installed
import findspark
findspark.init()
import pyspark

#Settings for PySpark to work
driver_memory = '4g'
num_executors = 2
executor_memory = '1g'
#pyspark_submit_args = ' --driver-memory ' + driver_memory + ' --executor-memory ' + executor_memory + ' --num-executors ' + num_executors + ' pyspark-shell'
pyspark_submit_args = ' --driver-memory ' + driver_memory + ' pyspark-shell'

#Setting the required parameters to start up PySpark
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

#Import Modules Needed for PySpark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  20, truncate = True):
    if(truncate):
        pd.set_option('display.max_colwidth', 50)
    else:
        pd.set_option('display.max_colwidth', -1)
    pd.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pd.reset_option('display.max_rows')

In [4]:
#Creating a spark session
spark = SparkSession.builder.appName("Data Transformation").getOrCreate()

### Step 3 - Read the data

We will be using the WDICountry, WDISeries and WDIData files for all the activity below.

In [5]:
#Read the file into a Spark Data Frame
country = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("../data/WDICountry.csv")

country.count()

264

In [6]:
series = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("../data/WDISeries.csv")

series.count()

1593

In [7]:
indicators = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("../data/WDIData.csv")

indicators.count()

420024

### Step 4 - Transform the data

#### Transform the Country Dataset

In [8]:
countryDim = country \
    .select("2-alpha code", "Country Code", "Short Name", "Long Name", "Region", "Income Group") \
    .withColumnRenamed("2-alpha code", "country_iso_code") \
    .withColumnRenamed("Country Code", "wb_country_code") \
    .withColumnRenamed("Short Name", "country_name") \
    .withColumnRenamed("Long Name", "country_long_name") \
    .withColumnRenamed("Region", "region") \
    .withColumnRenamed("Income Group", "income_group")
    
showDF(countryDim)

,country_iso_code,wb_country_code,country_name,country_long_name,region,income_group
0,AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
1,AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
2,AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
3,AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
4,AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
5,1A,ARB,Arab World,Arab World,None,None
6,AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
7,AR,ARG,Argentina,Argentine Republic,None,None
8,AR,The World Bank systematically assesses the app...,the World Bank has found that during 2012-201...,Upper middle income,Value added at basic prices (VAB),IBRD
9,AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income


#### You can also do transformations in sql
Let's rename the country_name and country_long_name columns

In [9]:
countryDim.createOrReplaceTempView("country")

In [10]:
transformQuery = """
select 
    country_iso_code
    , wb_country_code
    , country_name as name
    , country_long_name as long_name
    , region
    , income_group
from 
    country
"""

showDF(spark.sql(transformQuery))

,country_iso_code,wb_country_code,name,long_name,region,income_group
0,AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
1,AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
2,AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
3,AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
4,AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
5,1A,ARB,Arab World,Arab World,None,None
6,AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
7,AR,ARG,Argentina,Argentine Republic,None,None
8,AR,The World Bank systematically assesses the app...,the World Bank has found that during 2012-201...,Upper middle income,Value added at basic prices (VAB),IBRD
9,AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income


### Step 5 - Check the data quality

##### Do all countries have 2 character country_iso_codes ?

In [11]:
showDF(countryDim.select(col("country_iso_code"), length(col("country_iso_code")).alias("column_length")) \
    .groupBy("column_length").agg(count("*").alias("cnt")).filter("cnt > 1"))

,column_length,cnt
0,2,263


##### Do we have duplicate records for any of the key columns ?

In [12]:
showDF(countryDim.groupBy("country_iso_code").agg(count("*").alias("cnt")).filter("cnt > 1"))

showDF(countryDim.groupBy("wb_country_code").agg(count("*").alias("cnt")).filter("cnt > 1"))

showDF(countryDim.groupBy("country_name").agg(count("*").alias("cnt")).filter("cnt > 1"))

,country_iso_code,cnt
0,AR,2


,wb_country_code,cnt


,country_name,cnt


### Step 6 - Fix any issues with data quality

In [13]:
showDF(countryDim.filter("country_iso_code = 'AR'"))

,country_iso_code,wb_country_code,country_name,country_long_name,region,income_group
0,AR,ARG,Argentina,Argentine Republic,None,None
1,AR,The World Bank systematically assesses the app...,the World Bank has found that during 2012-201...,Upper middle income,Value added at basic prices (VAB),IBRD


In [14]:
countryDimFinal = countryDim.filter(length(col("wb_country_code")) == 3)

showDF(countryDimFinal)

,country_iso_code,wb_country_code,country_name,country_long_name,region,income_group
0,AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
1,AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
2,AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
3,AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
4,AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
5,1A,ARB,Arab World,Arab World,None,None
6,AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
7,AR,ARG,Argentina,Argentine Republic,None,None
8,AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income
9,AS,ASM,American Samoa,American Samoa,East Asia & Pacific,Upper middle income


### Step 7 - Write the data to the destination

In [15]:
# Here we are going to write the country dimension to an output csv file
countryDimFinal \
    .coalesce(1) \
    .write.csv('../output/CountryDim', mode='overwrite', header='true')

### Step 8 - Check if the output was written out as expected

In [16]:
%%bash
cat ../output/CountryDim/*csv | head

country_iso_code,wb_country_code,country_name,country_long_name,region,income_group
AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
1A,ARB,Arab World,Arab World,,
AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
AR,ARG,Argentina,Argentine Republic,,
AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income


## Excercise

#### Transform the Series Dataset
- Filter only for series that have Annual periodicity
- Get the following columns and rename the selected columns to prepare further processing  

| Name in CSV | Column Name |
| ------------- |:-------------:|
| Series Code | indicator_code |
| Indicator Name | indicator_name |
| Periodicity | periodicity |
| Aggregation Method | aggregation_method |


##### How many series do you end up with ?

In [17]:
seriesDim = series \
    .select("Series Code", "Indicator Name", "Short Definition", "Periodicity", "Aggregation Method") \
    .withColumnRenamed("Series Code", "indicator_code") \
    .withColumnRenamed("Indicator Name", "indicator_name") \
    .withColumnRenamed("Periodicity", "periodicity") \
    .withColumnRenamed("Aggregation Method", "aggregation_method") \
    .filter(col("periodicity") == "Annual") 
    
showDF(seriesDim)

seriesDim.count()

,indicator_code,indicator_name,Short Definition,periodicity,aggregation_method
0,AG.AGR.TRAC.NO,"Agricultural machinery, tractors",None,Annual,Sum
1,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...,None,Annual,Weighted average
2,AG.CON.FERT.ZS,Fertilizer consumption (kilograms per hectare ...,None,Annual,Weighted average
3,AG.LND.AGRI.K2,Agricultural land (sq. km),None,Annual,Sum
4,AG.LND.AGRI.ZS,Agricultural land (% of land area),None,Annual,Weighted average
5,AG.LND.ARBL.HA,Arable land (hectares),None,Annual,None
6,AG.LND.ARBL.HA.PC,Arable land (hectares per person),None,Annual,Weighted Average
7,AG.LND.ARBL.ZS,Arable land (% of land area),None,Annual,Weighted average
8,AG.LND.CREL.HA,Land under cereal production (hectares),None,Annual,Sum
9,AG.LND.CROP.ZS,Permanent cropland (% of land area),None,Annual,Weighted average


1587

#### What data quality checks were you able to perform ?

- Are there any duplicate codes ?
- Are all indicator_codes following the same pattern ?
- Is the case on columns consistent ? 

### Complex Transformations
##### Problem Statement
We want to measure the cellular and broadband penetration in comparison to the population demographics for every country.  It'll also be helpful to get some insights on annual global aggregates.

###### Our dataset has multiple types of metrics.  The only ones that we care about are simple aggregates.

In [18]:
simpleAggIndicators = seriesDim \
    .filter("lower(aggregation_method) = 'sum'") \
    .select("indicator_code", "indicator_name") \
    .orderBy("indicator_code")

showDF(simpleAggIndicators, limitRows = 500, truncate = False)

,indicator_code,indicator_name
0,AG.AGR.TRAC.NO,"Agricultural machinery, tractors"
1,AG.LND.AGRI.K2,Agricultural land (sq. km)
2,AG.LND.CREL.HA,Land under cereal production (hectares)
3,AG.LND.EL5M.RU.K2,Rural land area where elevation is below 5 meters (sq. km)
4,AG.LND.EL5M.UR.K2,Urban land area where elevation is below 5 meters (sq. km)
5,AG.LND.FRST.K2,Forest area (sq. km)
6,AG.LND.TOTL.K2,Land area (sq. km)
7,AG.LND.TOTL.RU.K2,Rural land area (sq. km)
8,AG.LND.TOTL.UR.K2,Urban land area (sq. km)
9,AG.PRD.CREL.MT,Cereal production (metric tons)


##### Only keep the indicators that are relevant to requirements i.e. Population indicators and Cellular and Broadband penetration

In [19]:
targetIndicators = simpleAggIndicators \
    .filter("lower(indicator_name) like '%population%total%' " + 
            " or lower(indicator_name) like '%cellular%' " +
            " or lower(indicator_name) like '%broadband%'") \
    .filter("lower(indicator_name) not like '%refugee%'")

showDF(targetIndicators)

,indicator_code,indicator_name
0,IT.CEL.SETS,Mobile cellular subscriptions
1,IT.NET.BBND,Fixed broadband subscriptions
2,SP.POP.0014.TO,"Population ages 0-14, total"
3,SP.POP.1564.TO,"Population ages 15-64, total"
4,SP.POP.65UP.TO,"Population ages 65 and above, total"
5,SP.POP.TOTL,"Population, total"


##### Now that we have identified the various indicators of interest, we can continue with getting the metrics for these indicators

In [20]:
# Keep the columns that are relevant for further transformations
indicatorsData = indicators \
    .withColumnRenamed("Indicator Code", "indicator_code") \
    .withColumnRenamed("Country Code", "wb_country_code") \
    .drop("Indicator Name") \
    .drop("Country Name") \
    .drop("_c62")

In [21]:
#Keep only the indicators that we care about
targetIndicatorsData = indicatorsData.join(targetIndicators \
                                         , indicatorsData.indicator_code == targetIndicators.indicator_code) \
    .drop(targetIndicators.indicator_code)

In [22]:
showDF(targetIndicatorsData)

,wb_country_code,indicator_code,1960,1961,1962,1963,1964,1965,1966,1967,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,indicator_name
0,ARB,IT.NET.BBND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.845325e+06,7.123797e+06,8.570787e+06,1.032359e+07,1.179999e+07,1.383407e+07,1.660445e+07,1.852699e+07,None,Fixed broadband subscriptions
1,ARB,IT.CEL.SETS,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,2.652708e+08,3.122478e+08,3.519580e+08,3.816419e+08,4.077045e+08,4.150294e+08,4.171956e+08,4.123153e+08,None,Mobile cellular subscriptions
2,ARB,SP.POP.0014.TO,40064255.0,41518715.0,42987169.0,44458639.0,45910606.0,47331020.0,48923976.0,50456294.0,...,1.180752e+08,1.199217e+08,1.221659e+08,1.246738e+08,1.273441e+08,1.299870e+08,1.324654e+08,1.350077e+08,None,"Population ages 0-14, total"
3,ARB,SP.POP.1564.TO,49179481.0,50158016.0,51208328.0,52348368.0,53610121.0,55012095.0,56343748.0,57859656.0,...,2.156395e+08,2.217716e+08,2.274856e+08,2.329511e+08,2.382154e+08,2.434054e+08,2.486149e+08,2.535661e+08,None,"Population ages 15-64, total"
4,ARB,SP.POP.65UP.TO,3247196.0,3367766.0,3486797.0,3604069.0,3719175.0,3831873.0,3962869.0,4090982.0,...,1.443047e+07,1.481554e+07,1.524433e+07,1.568205e+07,1.614261e+07,1.665064e+07,1.722468e+07,1.787893e+07,None,"Population ages 65 and above, total"
5,ARB,SP.POP.TOTL,92490932.0,95044497.0,97682294.0,100411076.0,103239902.0,106174988.0,109230593.0,112406932.0,...,3.481451e+08,3.565089e+08,3.648959e+08,3.733070e+08,3.817021e+08,3.900430e+08,3.983050e+08,4.064527e+08,None,"Population, total"
6,CSS,IT.NET.BBND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.341620e+05,4.731380e+05,4.982850e+05,6.272510e+05,6.744190e+05,7.533680e+05,8.971340e+05,9.818860e+05,None,Fixed broadband subscriptions
7,CSS,IT.CEL.SETS,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,7.646708e+06,7.935923e+06,7.534114e+06,7.322041e+06,7.738401e+06,7.952805e+06,8.158698e+06,8.487436e+06,None,Mobile cellular subscriptions
8,CSS,SP.POP.0014.TO,1766884.0,1816372.0,1864013.0,1908732.0,1949651.0,1986537.0,2027526.0,2065067.0,...,1.828309e+06,1.804177e+06,1.783918e+06,1.761456e+06,1.738418e+06,1.717638e+06,1.700765e+06,1.687271e+06,None,"Population ages 0-14, total"
9,CSS,SP.POP.1564.TO,2151060.0,2173846.0,2198714.0,2225619.0,2253715.0,2281832.0,2303230.0,2324023.0,...,4.477802e+06,4.533534e+06,4.585728e+06,4.638828e+06,4.691000e+06,4.739145e+06,4.781112e+06,4.816676e+06,None,"Population ages 15-64, total"


#### The output that we see currently isn't the most ideal from a modelling perspective.  Ideally we would like every row to look like the output of the following code block in the new dataset that we plan to create.

In [23]:
indicatorsSample = targetIndicatorsData \
    .select(col("wb_country_code")
            , col("indicator_code")
            , lit("1960").alias("year")
            , col("1960").alias("indicator_value")) \
    .filter("indicator_value >= 0.0")

showDF(indicatorsSample)

,wb_country_code,indicator_code,year,indicator_value
0,ARB,IT.CEL.SETS,1960,0.0
1,ARB,SP.POP.0014.TO,1960,40064255.0
2,ARB,SP.POP.1564.TO,1960,49179481.0
3,ARB,SP.POP.65UP.TO,1960,3247196.0
4,ARB,SP.POP.TOTL,1960,92490932.0
5,CSS,IT.CEL.SETS,1960,0.0
6,CSS,SP.POP.0014.TO,1960,1766884.0
7,CSS,SP.POP.1564.TO,1960,2151060.0
8,CSS,SP.POP.65UP.TO,1960,169157.0
9,CSS,SP.POP.TOTL,1960,4198307.0


##### Let us start by getting the list of years that we have metrics for

In [24]:
yearList = [x for x in targetIndicatorsData.schema.names \
             if x != 'wb_country_code' and x != 'indicator_code' and x != 'indicator_name'] 

print(yearList)

['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']


In [25]:
#Cheat for creating a dataframe with no rows 
indicatorsDF = indicatorsSample.filter('1 = 0')

#Iterate through the list of years and store the rows in the DataFrame we created above
for indicatorYear in yearList:
    print("Processing indicators for " + indicatorYear)
    yearIndicatorDF = targetIndicatorsData \
        .select(col("wb_country_code")
                , col("indicator_code")
                , lit(indicatorYear).alias("year")
                , col(indicatorYear).alias("indicator_value")) \
        .filter("indicator_value >= 0")
    indicatorsDF = indicatorsDF.union(yearIndicatorDF)    

Processing indicators for 1960
Processing indicators for 1961
Processing indicators for 1962
Processing indicators for 1963
Processing indicators for 1964
Processing indicators for 1965
Processing indicators for 1966
Processing indicators for 1967
Processing indicators for 1968
Processing indicators for 1969
Processing indicators for 1970
Processing indicators for 1971
Processing indicators for 1972
Processing indicators for 1973
Processing indicators for 1974
Processing indicators for 1975
Processing indicators for 1976
Processing indicators for 1977
Processing indicators for 1978
Processing indicators for 1979
Processing indicators for 1980
Processing indicators for 1981
Processing indicators for 1982
Processing indicators for 1983
Processing indicators for 1984
Processing indicators for 1985
Processing indicators for 1986
Processing indicators for 1987
Processing indicators for 1988
Processing indicators for 1989
Processing indicators for 1990
Processing indicators for 1991
Processi

#### Let's cache the dataset to iterate over it

In [26]:
indicatorsDF.cache()

#Force the data to be cached
indicatorsDF.count()

70303

In [27]:
#Check the indicator counts per year
#showDF(indicatorsDF.groupBy('year').agg(count("*")).orderBy("year"), limitRows=100)

#### Getting yearly indicator totals

In [28]:
yearPivot = indicatorsDF.groupBy('year').pivot('indicator_code').sum('indicator_value') 

In [29]:
showDF(yearPivot.orderBy('year'))

,year,IT.CEL.SETS,IT.NET.BBND,SP.POP.0014.TO,SP.POP.1564.TO,SP.POP.65UP.TO,SP.POP.TOTL
0,1960,0.0,None,1.164043e+10,1.764656e+10,1.419169e+09,3.072052e+10
1,1961,NaN,None,1.186066e+10,1.780707e+10,1.451321e+09,3.113369e+10
2,1962,NaN,None,1.215358e+10,1.803340e+10,1.483492e+09,3.168550e+10
3,1963,NaN,None,1.249841e+10,1.834110e+10,1.517104e+09,3.237204e+10
4,1964,NaN,None,1.281722e+10,1.868178e+10,1.549219e+09,3.306404e+10
5,1965,0.0,None,1.310455e+10,1.907143e+10,1.581083e+09,3.377327e+10
6,1966,NaN,None,1.343731e+10,1.943187e+10,1.634468e+09,3.452025e+10
7,1967,NaN,None,1.371767e+10,1.984342e+10,1.687816e+09,3.526591e+10
8,1968,NaN,None,1.396924e+10,2.029706e+10,1.741146e+09,3.602484e+10
9,1969,NaN,None,1.422516e+10,2.078273e+10,1.795141e+09,3.682082e+10


In [30]:
yearPivot.printSchema()

root
 |-- year: string (nullable = false)
 |-- IT.CEL.SETS: double (nullable = true)
 |-- IT.NET.BBND: double (nullable = true)
 |-- SP.POP.0014.TO: double (nullable = true)
 |-- SP.POP.1564.TO: double (nullable = true)
 |-- SP.POP.65UP.TO: double (nullable = true)
 |-- SP.POP.TOTL: double (nullable = true)



In [31]:
yearPivotDF = yearPivot.orderBy('year') \
    .withColumnRenamed('IT.CEL.SETS', 'cellular_subscriptions') \
    .withColumnRenamed('IT.NET.BBND', 'broadband_subscriptions') \
    .withColumnRenamed('SP.POP.0014.TO', 'population_age_0_to_14') \
    .withColumnRenamed('SP.POP.1564.TO', 'population_age_15_64') \
    .withColumnRenamed('SP.POP.65UP.TO', 'population_age_65_and_above') \
    .withColumnRenamed('SP.POP.TOTL', 'population')

#### Data Quality Checkpoint 

In [32]:
yearPivotDF.cache()

DataFrame[year: string, cellular_subscriptions: double, broadband_subscriptions: double, population_age_0_to_14: double, population_age_15_64: double, population_age_65_and_above: double, population: double]

In [33]:
yearPivotDF.filter('population_age_0_to_14 < 0').count()

0

In [34]:
yearPivotDF.filter('population_age_15_64 < 0').count()

0

In [35]:
yearPivotDF.filter('population_age_0_to_14 < 0').count()

0

In [36]:
yearPivotDF.filter('population_age_65_and_above < 0').count()

0

In [37]:
yearPivotDF.filter('population < 0').count()

0

In [38]:
yearPivotDF.filter('cellular_subscriptions < 0').count()

0

In [39]:
yearPivotDF.filter('broadband_subscriptions < 0').count()

0

In [40]:
yearPivotDF.filter('population_age_0_to_14 > population').count()

0

In [41]:
yearPivotDF.filter('population_age_15_64 > population').count()

0

In [42]:
yearPivotDF.filter('population_age_65_and_above > population').count()

0

In [43]:
yearPivotDF.filter('(population_age_0_to_14 + population_age_15_64 + population_age_65_and_above) > population').count()

0

In [44]:
#Write the yearly totals to a CSV File
yearPivotDF \
    .select(col('year')
            , col('population').cast(DecimalType(38, 2))
            , col('population_age_0_to_14').cast(DecimalType(38, 2))
            , col('population_age_15_64').cast(DecimalType(38, 2))
            , col('population_age_65_and_above').cast(DecimalType(38, 2))
            , col('broadband_subscriptions').cast(DecimalType(38, 2))
            , col('cellular_subscriptions').cast(DecimalType(38, 2))) \
    .coalesce(1) \
    .write.csv('../output/YearlyStats', mode='overwrite', header='true')

#### Getting yearly regional totals

In [45]:
regionalIndicators = indicatorsDF.join(countryDimFinal
                                       , indicatorsDF.wb_country_code == countryDim.wb_country_code
                                       , "inner") \
    .select(countryDim.region
            , indicatorsDF.wb_country_code
            , indicatorsDF.year
            , indicatorsDF.indicator_code
            , indicatorsDF.indicator_value)

In [46]:
showDF(regionalIndicators)

,region,wb_country_code,year,indicator_code,indicator_value
0,None,ARB,1960,IT.CEL.SETS,0.0
1,None,ARB,1960,SP.POP.0014.TO,40064255.0
2,None,ARB,1960,SP.POP.1564.TO,49179481.0
3,None,ARB,1960,SP.POP.65UP.TO,3247196.0
4,None,ARB,1960,SP.POP.TOTL,92490932.0
5,None,CSS,1960,IT.CEL.SETS,0.0
6,None,CSS,1960,SP.POP.0014.TO,1766884.0
7,None,CSS,1960,SP.POP.1564.TO,2151060.0
8,None,CSS,1960,SP.POP.65UP.TO,169157.0
9,None,CSS,1960,SP.POP.TOTL,4198307.0


In [47]:
regionalPivot = regionalIndicators.groupBy('region', 'year').pivot('indicator_code').sum('indicator_value')

In [48]:
showDF(regionalPivot.orderBy('region', 'year'), limitRows=100)

,region,year,IT.CEL.SETS,IT.NET.BBND,SP.POP.0014.TO,SP.POP.1564.TO,SP.POP.65UP.TO,SP.POP.TOTL
0,None,1960,0.000000e+00,NaN,1.052918e+10,1.591379e+10,1.270274e+09,2.772620e+10
1,None,1961,NaN,NaN,1.072883e+10,1.605789e+10,1.299015e+09,2.809895e+10
2,None,1962,NaN,NaN,1.099508e+10,1.626195e+10,1.327810e+09,2.859841e+10
3,None,1963,NaN,NaN,1.130876e+10,1.653985e+10,1.357916e+09,2.922045e+10
4,None,1964,NaN,NaN,1.159883e+10,1.684778e+10,1.386660e+09,2.984755e+10
5,None,1965,0.000000e+00,NaN,1.186024e+10,1.720040e+10,1.415175e+09,3.049045e+10
6,None,1966,NaN,NaN,1.216311e+10,1.752669e+10,1.463169e+09,3.116795e+10
7,None,1967,NaN,NaN,1.241838e+10,1.789958e+10,1.511139e+09,3.184446e+10
8,None,1968,NaN,NaN,1.264760e+10,1.831104e+10,1.559115e+09,3.253346e+10
9,None,1969,NaN,NaN,1.288063e+10,1.875102e+10,1.607638e+09,3.325536e+10


In [49]:
#Write the regional-yearly totals to a CSV File
regionalPivot.filter('region is not null') \
    .orderBy('region','year') \
    .withColumnRenamed('IT.CEL.SETS', 'cellular_subscriptions') \
    .withColumnRenamed('IT.NET.BBND', 'broadband_subscriptions') \
    .withColumnRenamed('SP.POP.0014.TO', 'population_age_0_to_14') \
    .withColumnRenamed('SP.POP.1564.TO', 'population_age_15_64') \
    .withColumnRenamed('SP.POP.65UP.TO', 'population_age_65_and_above') \
    .withColumnRenamed('SP.POP.TOTL', 'population') \
    .select(col('region')
            , col('year')
            , col('population').cast(DecimalType(38, 2))
            , col('population_age_0_to_14').cast(DecimalType(38, 2))
            , col('population_age_15_64').cast(DecimalType(38, 2))
            , col('population_age_65_and_above').cast(DecimalType(38, 2))
            , col('broadband_subscriptions').cast(DecimalType(38, 2))
            , col('cellular_subscriptions').cast(DecimalType(38, 2))) \
    .coalesce(1) \
    .write.csv('../output/RegionalStats', mode='overwrite', header='true')

## Short Excercise

Becky finds the regional metrics interesting, but she wants to look at these metrics at a country level for each year.  Can you adapt the regional pivot that we computed earlier to get the metrics for each year and country ?

In [50]:
countryIndicators = indicatorsDF.join(countryDimFinal
                                       , indicatorsDF.wb_country_code == countryDim.wb_country_code
                                       , "inner") \
    .select(indicatorsDF.wb_country_code
            , countryDim.country_iso_code
            , countryDim.country_name
            , indicatorsDF.year
            , indicatorsDF.indicator_code
            , indicatorsDF.indicator_value)

showDF(countryIndicators)

,wb_country_code,country_iso_code,country_name,year,indicator_code,indicator_value
0,ARB,1A,Arab World,1960,IT.CEL.SETS,0.0
1,ARB,1A,Arab World,1960,SP.POP.0014.TO,40064255.0
2,ARB,1A,Arab World,1960,SP.POP.1564.TO,49179481.0
3,ARB,1A,Arab World,1960,SP.POP.65UP.TO,3247196.0
4,ARB,1A,Arab World,1960,SP.POP.TOTL,92490932.0
5,CSS,S3,Caribbean small states,1960,IT.CEL.SETS,0.0
6,CSS,S3,Caribbean small states,1960,SP.POP.0014.TO,1766884.0
7,CSS,S3,Caribbean small states,1960,SP.POP.1564.TO,2151060.0
8,CSS,S3,Caribbean small states,1960,SP.POP.65UP.TO,169157.0
9,CSS,S3,Caribbean small states,1960,SP.POP.TOTL,4198307.0


In [51]:
countryPivot = countryIndicators.groupBy('country_iso_code', 'country_name', 'year') \
    .pivot('indicator_code').sum('indicator_value')

In [52]:
showDF(countryPivot.orderBy('country_iso_code', 'country_name', 'year'), limitRows=100)

,country_iso_code,country_name,year,IT.CEL.SETS,IT.NET.BBND,SP.POP.0014.TO,SP.POP.1564.TO,SP.POP.65UP.TO,SP.POP.TOTL
0,None,Channel Islands,1960,NaN,None,23546.0,71467.0,14407.0,109420.0
1,None,Channel Islands,1961,NaN,None,23857.0,71951.0,14591.0,110399.0
2,None,Channel Islands,1962,NaN,None,23955.0,72685.0,14817.0,111457.0
3,None,Channel Islands,1963,NaN,None,24103.0,73419.0,15073.0,112595.0
4,None,Channel Islands,1964,NaN,None,24399.0,74038.0,15336.0,113773.0
5,None,Channel Islands,1965,NaN,None,24800.0,74587.0,15608.0,114995.0
6,None,Channel Islands,1966,NaN,None,25025.0,75355.0,15847.0,116227.0
7,None,Channel Islands,1967,NaN,None,25131.0,76242.0,16101.0,117474.0
8,None,Channel Islands,1968,NaN,None,25373.0,77002.0,16351.0,118726.0
9,None,Channel Islands,1969,NaN,None,25790.0,77576.0,16606.0,119972.0


In [53]:
#Write the regional-yearly totals to a CSV File
countryPivot.filter('country_iso_code is not null') \
    .orderBy('country_iso_code','country_name', 'year') \
    .withColumnRenamed('IT.CEL.SETS', 'cellular_subscriptions') \
    .withColumnRenamed('IT.NET.BBND', 'broadband_subscriptions') \
    .withColumnRenamed('SP.POP.0014.TO', 'population_age_0_to_14') \
    .withColumnRenamed('SP.POP.1564.TO', 'population_age_15_64') \
    .withColumnRenamed('SP.POP.65UP.TO', 'population_age_65_and_above') \
    .withColumnRenamed('SP.POP.TOTL', 'population') \
    .select(col('country_iso_code')
            , col('country_name')
            , col('year')
            , col('population').cast(DecimalType(38, 2))
            , col('population_age_0_to_14').cast(DecimalType(38, 2))
            , col('population_age_15_64').cast(DecimalType(38, 2))
            , col('population_age_65_and_above').cast(DecimalType(38, 2))
            , col('broadband_subscriptions').cast(DecimalType(38, 2))
            , col('cellular_subscriptions').cast(DecimalType(38, 2))) \
    .coalesce(1) \
    .write.csv('../output/CountryStats', mode='overwrite', header='true')

## Excercise

Kat wants to identify the countries that are conducive to start a business.  She thinks that it would suffice to look at the most recent metrics for the following:
- Gross National Income (GNI)
- Cost of business start-up procedures
- Number of days required to start a business
- Number of start-up procedures to register a business
- GDP
- GDP per capita
- Business Regulatory Environment
- Ease of doing business index (Only available in 2017)

Write the data to a csv file.

Hint - Start by matching up indicators that might have descriptions matching what Kat is looking for. e.g. <br>
`showDF(seriesDim.filter("indicator_name like '%GNI%'").orderBy("indicator_code"), limitRows = 500)`

In [54]:
recentIndicators = indicatorsData \
    .select("wb_country_code", "indicator_code", "2016") \
    .filter(col('indicator_code').isin('IC.REG.COST.PC.ZS', 'IC.REG.DURS', 'IC.REG.PROC', \
        'NY.GNP.ATLS.CD', 'NY.GDP.MKTP.KD', 'NY.GDP.PCAP.KD', 'IQ.CPA.BREG.XQ', 'IC.BUS.EASE.XQ')) \
    .withColumnRenamed("2016", "indicator_value") \
    .withColumn("indicator_value", col("indicator_value").cast(DecimalType(38, 2)))

showDF(recentIndicators)

,wb_country_code,indicator_code,indicator_value
0,ARB,IC.REG.COST.PC.ZS,36.47
1,ARB,IQ.CPA.BREG.XQ,2.80
2,ARB,IC.BUS.EASE.XQ,None
3,ARB,NY.GDP.MKTP.KD,2616760000000.00
4,ARB,NY.GDP.PCAP.KD,6438.05
5,ARB,NY.GNP.ATLS.CD,2676040000000.00
6,ARB,IC.REG.PROC,8.32
7,ARB,IC.REG.DURS,23.17
8,CSS,IC.REG.COST.PC.ZS,19.90
9,CSS,IQ.CPA.BREG.XQ,3.90


In [55]:
businessIndexIndicators = indicatorsData \
    .select("wb_country_code", "indicator_code", "2017") \
    .filter(col('indicator_code').isin('IC.BUS.EASE.XQ')) \
    .withColumnRenamed("2017", "indicator_value") \
    .withColumn("indicator_value", col("indicator_value").cast(DecimalType(38, 2)))

In [56]:
allIndicators = recentIndicators.union(businessIndexIndicators)

In [57]:
countryBusinessStartupPivot = allIndicators.join(countryDimFinal
                                       , recentIndicators.wb_country_code == countryDim.wb_country_code
                                       , "inner") \
    .select(countryDimFinal.country_iso_code
            , countryDimFinal.country_name
            , recentIndicators.indicator_code
            , recentIndicators.indicator_value) \
    .groupBy('country_iso_code', 'country_name').pivot('indicator_code').sum('indicator_value') \
    .withColumnRenamed('country_iso_code', 'Country ISO Code') \
    .withColumnRenamed('country_name', 'Country Name') \
    .withColumnRenamed('NY.GNP.ATLS.CD', 'GNI') \
    .withColumnRenamed('IC.REG.DURS', 'Startup Time') \
    .withColumnRenamed('IC.REG.PROC', 'Startup Procedures') \
    .withColumnRenamed('IC.REG.COST.PC.ZS', 'Startup Cost Pct of GNI') \
    .withColumnRenamed('NY.GDP.MKTP.KD', 'GDP') \
    .withColumnRenamed('NY.GDP.PCAP.KD', 'GDP Per Capita') \
    .withColumnRenamed('IQ.CPA.BREG.XQ', 'Business Regulation') \
    .withColumnRenamed('IC.BUS.EASE.XQ', 'Ease of business') \
    .withColumn('Startup Cost', (col('GNI') * col('Startup Cost Pct of GNI') / lit(100.0)).cast(DecimalType(38, 2))) \
    .filter(col('GNI') > 0) \
    .filter(col('Startup Time').isNotNull()) \
    .filter(col('Startup Procedures').isNotNull()) \
    .filter(col('Startup Cost').isNotNull()) 

In [58]:
showDF(countryBusinessStartupPivot, limitRows = 500)

,Country ISO Code,Country Name,Ease of business,Startup Cost Pct of GNI,Startup Time,Startup Procedures,Business Regulation,GDP,GDP Per Capita,GNI,Startup Cost
0,BJ,Benin,151.00,3.80,8.50,6.00,3.00,9103831278.00,837.34,8938879520.00,339677421.76
1,XC,Euro area,None,3.83,9.55,5.26,None,13377100000000.00,39256.27,12312600000000.00,471572580000.00
2,KZ,Kazakhstan,36.00,0.30,9.00,5.00,None,188309000000.00,10582.50,156812000000.00,470436000.00
3,JM,Jamaica,70.00,4.30,10.00,2.00,None,13801803130.00,4790.04,13349534142.00,574029968.11
4,NO,Norway,8.00,0.90,4.00,4.00,None,472766000000.00,90288.82,429276000000.00,3863484000.00
5,IR,Iran,124.00,1.20,15.50,9.00,None,540581000000.00,6733.91,438869000000.00,5266428000.00
6,AG,Antigua and Barbuda,107.00,9.40,22.00,9.00,None,1344373698.00,13315.51,1369440563.00,128727412.92
7,CG,Congo,179.00,61.20,50.00,11.00,2.50,14342385958.00,2798.07,8770006555.00,5367244011.66
8,AR,Argentina,117.00,10.80,24.00,13.00,None,445227000000.00,10154.00,524974000000.00,56697192000.00
9,HT,Haiti,181.00,219.30,97.00,12.00,2.00,7910618370.00,729.27,8488280524.00,18614799189.13


In [59]:
countryBusinessStartupPivot \
    .select("Country ISO Code", "Country Name", "GDP", "GDP Per Capita", "GNI", \
            "Startup Cost", "Startup Cost Pct of GNI", "Startup Time", "Startup Procedures", \
            "Business Regulation", "Ease of business") \
    .coalesce(1) \
    .write.csv('../output/BusinessStartupData', mode='overwrite', header='true')